In [64]:
import requests
import pandas as pd

In [101]:
def get_tarifas_distribuidoras(CNPJ):
    final_df = pd.DataFrame()
    base_url = "https://dadosabertos.aneel.gov.br"
    url = f"{base_url}/api/3/action/datastore_search?resource_id=fcf2906c-7c32-4b9b-a637-054e7a5234f4&q={CNPJ}"
    
    while url:
        data = requests.get(url).json()
        df = pd.json_normalize(data["result"]["records"])
        
        if len(df) > 0:
            final_df = pd.concat([final_df, df], axis=0)
            url = base_url + data["result"]["_links"]["next"]
        else:
            url = None

    return final_df

In [115]:
def trata_df_tarifas(df):
    df["DatInicioVigencia"] = pd.to_datetime(df["DatInicioVigencia"])
    filter_DscSubGrupo = df["DscSubGrupo"] == "B1"
    filter_DscModalidadeTarifaria = df["DscModalidadeTarifaria"] == "Convencional"
    filter_DscSubClasse = df["DscSubClasse"] == "Residencial"
    filter_DscBaseTarifaria = df["DscBaseTarifaria"] == "Base Econômica"
    filter_DatInicioVigencia = df["DatInicioVigencia"] == df["DatInicioVigencia"].max()
    return df[filter_DscSubGrupo & filter_DscModalidadeTarifaria & filter_DscSubClasse & filter_DscBaseTarifaria & filter_DatInicioVigencia]

In [136]:
empresas = {
    "Light": "60444437000146",
    "Enel RJ": "33050071000158"
}

dataframes = []
for empresa in empresas.keys():
    dataframes.append(get_tarifas_distribuidoras(empresas[empresa]).pipe(trata_df_tarifas))

df_tarifas = pd.concat(dataframes)

In [137]:
df_tarifas

,_id,DatGeracaoConjuntoDados,DscREH,SigAgente,NumCNPJDistribuidora,DatInicioVigencia,DatFimVigencia,DscBaseTarifaria,DscSubGrupo,DscModalidadeTarifaria,DscClasse,DscSubClasse,DscDetalhe,NomPostoTarifario,DscUnidadeTerciaria,SigAgenteAcessante,VlrTUSD,VlrTE,rank
63,67736,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.014, DE 15 DE MAR...",LIGHT,60444437000146,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,"463,33","346,57",0.057309
42,70927,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.015, DE 15 DE MAR...",ENEL RJ,33050071000158,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,"523,52","307,44",0.057309


In [139]:
df_tarifas.to_csv("../dados/TarifasDistribuidoras.csv")

In [141]:
dados_vendedores = pd.read_csv("../dados/ConsolidadoVendedoresLeilao.csv",sep=';',encoding='mbcs')

In [134]:
dados_vendedores.head()

,ID Resultado Vendedor,Ano do Leilão,Tipo Leilão,Leilão,Denominação do Leilão,Edital do Leilão ANEEL,Sigla Vendedor,CNPJ,Produto,CEG,...,ICE (R$/MWh),CVU (R$/MWh),Montante Financeiro Data Leilão (R$),Preço de Venda Atualizado (R$/MWh),Montante Financeiro Atualizado (R$),Início Suprimento,Fim Suprimento,Entrega Escalonada,IPCA na Data do leilão,IPCA Abr/2022
0,1922,2021,LEE,23ºLEE,Leilão de Energia Existente,004/2021,PETROBRAS PIE,33.000.167/0001-01,DISP2025-15,UTE.GN.SP.028014-3.01,...,"0,00","0,00","1.953.416.892,24","168,09","2.172.366.107,01",01/01/2025,31/12/2039,Não,"5.739,56","6.382,88"
1,1923,2021,LEE,24ºLEE,Leilão de Energia Existente,005/2021,PETROBRAS PIE,33.000.167/0001-01,DISP2026-15,UTE.GN.SP.028014-3.01,...,"0,00","0,00","1.455.323.827,25","191,71","1.618.444.157,82",01/01/2026,31/12/2040,Não,"5.739,56","6.382,88"
2,1924,2021,LEN,33ºLEN,Leilão de Energia Nova,006/2021,COPREL,08.323.274/0001-23,QTDH2024-30,PCH.PH.RS.037469-5.01,...,"0,00","0,00","206.396.121,60","241,16","228.319.972,80",01/01/2024,31/12/2053,Não,"5.769,98","6.382,88"
3,1925,2021,LEN,33ºLEN,Leilão de Energia Nova,006/2021,RINCAO S MIGUEL A4,08.147.432/0001-31,QTDH2024-30,PCH.PH.RS.031029-8.01,...,"0,00","0,00","241.546.580,35","241,91","267.204.190,79",01/01/2024,31/12/2053,Não,"5.769,98","6.382,88"
4,1926,2021,LEN,33ºLEN,Leilão de Energia Nova,006/2021,CACH 5 VEADOS A4,08.147.388/0001-60,QTDH2024-30,PCH.PH.RS.031030-1.01,...,"0,00","0,00","417.204.196,99","243,73","461.520.546,85",01/01/2024,31/12/2053,Não,"5.769,98","6.382,88"
